In [ ]:
from urllib import request, parse
from http.cookiejar import CookieJar
import numpy
import getpass
import netrc
import requests
import json
import boto3
import os
import boto3
import matplotlib.pyplot as plt
import io
from urllib.parse import urlparse
import sys  
!{sys.executable} -m pip install --user jsonlines
import jsonlines
from PIL import Image


In [ ]:
def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

In [ ]:
setup_earthdata_login_auth('urs.earthdata.nasa.gov')
#response = requests.get("https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials")
#creds = json.loads(response.content)

In [ ]:
session = boto3.Session(profile_name='sandbox')
client = session.client('s3')

counter=0

f = open("cloud_training.json",'w')
response = client.list_objects_v2(
    Bucket='dug-cloud-training',
    MaxKeys=200,
)

with jsonlines.Writer(f) as writer:
    client
    for key in response['Contents']:
        s3_loc = key['Key']
        if s3_loc.endswith('.jpg'):
            print(s3_loc)
            response = client.get_object(
                Bucket='dug-cloud-training',
                Key=s3_loc)
            # Render image
            file_stream = response['Body']
            img = Image.open(file_stream)
            imgplot = plt.imshow(img)
            plt.show(imgplot)       
            # Ask for classification           
            while True:
                try:
                    if counter < 100:
                        cloudy = int(input("Cloudy? [1:cloudy, 0:not cloudy]:"))
                        if cloudy < 0 or cloudy > 1:
                            raise ValueError          
                    data = {}
                    data['source-ref'] = "s3://dug-cloud-training/" + s3_loc
                    if counter < 100:
                        data['class'] = str(cloudy)
                    counter=counter+1
                    writer.write(data)        
                    break
                except ValueError:
                    print("Invalid input. The number must be in the range of 0-1.")                  
f.close()
print('Done!')